<a href="https://colab.research.google.com/github/SabadModi/RecipeRecommender/blob/main/RecipeReccomendationUsingIngredients.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Dataset

In [ ]:
!pip install kaggle
!pip install gensim==4.1.2

     |████████████████████████████████| 24.1 MB 84.8 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
!mkdir ~/.kaggle
!mkdir dataset

In [ ]:
# Upload your Kaggle API JSON file
from google.colab import files

files.upload()

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d shuyangli94/food-com-recipes-and-user-interactions

 96% 257M/267M [00:02<00:00, 129MB/s]
100% 267M/267M [00:02<00:00, 116MB/s]


In [ ]:
!unzip food-com-recipes-and-user-interactions.zip -d dataset/

Archive:  food-com-recipes-and-user-interactions.zip
replace dataset/PP_recipes.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dataset/PP_users.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dataset/RAW_interactions.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dataset/RAW_recipes.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: dataset/RAW_recipes.csv  
replace dataset/ingr_map.pkl? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dataset/interactions_test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dataset/interactions_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dataset/interactions_validation.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


# Preparing the data

In [ ]:
import pandas as pd
df_recipes = pd.read_csv("dataset/RAW_recipes.csv")

In [ ]:
df_recipes = df_recipes.drop(columns = ['minutes', 'id', 'contributor_id', 'submitted', 'nutrition', 'n_steps', 'steps', 'description', 'n_ingredients'])

In [ ]:
df_recipes

,name,tags,ingredients
0,arriba baked winter squash mexican style,"['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'side-dishes', 'vegetables', 'mexican', 'easy', 'fall', 'holiday-event', 'vegetarian', 'winter', 'dietary', 'christmas', 'seasonal', 'squash']","['winter squash', 'mexican seasoning', 'mixed spice', 'honey', 'butter', 'olive oil', 'salt']"
1,a bit different breakfast pizza,"['30-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'breakfast', 'main-dish', 'pork', 'american', 'oven', 'easy', 'kid-friendly', 'pizza', 'dietary', 'northeastern-united-states', 'meat', 'equipment']","['prepared pizza crust', 'sausage patty', 'eggs', 'milk', 'salt and pepper', 'cheese']"
2,all in the kitchen chili,"['time-to-make', 'course', 'preparation', 'main-dish', 'chili', 'crock-pot-slow-cooker', 'dietary', 'equipment', '4-hours-or-less']","['ground beef', 'yellow onions', 'diced tomatoes', 'tomato paste', 'tomato soup', 'rotel tomatoes', 'kidney beans', 'water', 'chili powder', 'ground cumin', 'salt', 'lettuce', 'cheddar cheese']"
3,alouette potatoes,"['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'preparation', 'occasion', 'side-dishes', 'eggs-dairy', 'potatoes', 'vegetables', 'oven', 'easy', 'dinner-party', 'holiday-event', 'easter', 'cheese', 'stove-top', 'dietary', 'christmas', 'new-years', 'thanksgiving', 'independence-day', 'st-patricks-day', 'valentines-day', 'inexpensive', 'brunch', 'superbowl', 'equipment', 'presentation', 'served-hot']","['spreadable cheese with garlic and herbs', 'new potatoes', 'shallots', 'parsley', 'tarragon', 'olive oil', 'red wine vinegar', 'salt', 'pepper', 'red bell pepper', 'yellow bell pepper']"
4,amish tomato ketchup for canning,"['weeknight', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'canning', 'condiments-etc', 'vegetables', 'american', 'heirloom-historical', 'holiday-event', 'vegetarian', 'dietary', 'amish-mennonite', 'northeastern-united-states', 'number-of-servings', 'technique', '4-hours-or-less']","['tomato juice', 'apple cider vinegar', 'sugar', 'salt', 'pepper', 'clove oil', 'cinnamon oil', 'dry mustard']"
...,...,...,...
231632,zydeco soup,"['ham', '60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'north-american', 'soups-stews', 'pork', 'vegetables', 'american', 'cajun', 'southern-united-states', 'easy', 'creole', 'meat', 'peppers', 'tomatoes', '3-steps-or-less']","['celery', 'onion', 'green sweet pepper', 'garlic cloves', 'olive oil', 'cooked ham', 'paprika', 'sugar', 'dry mustard', 'ground cumin', 'dried basil', 'dried oregano', 'dried thyme', 'ground cloves', 'black pepper', 'cayenne pepper', 'black-eyed peas', 'yellow hominy', 'diced tomatoes', 'low sodium chicken broth', 'fresh parsley', 'molasses']"
231633,zydeco spice mix,"['15-minutes-or-less', 'time-to-make', 'course', 'preparation', 'for-large-groups', 'condiments-etc', 'easy', 'vegan', 'vegetarian', 'herb-and-spice-mixes', 'dietary', 'cooking-mixes', 'number-of-servings', '3-steps-or-less']","['paprika', 'salt', 'garlic powder', 'onion powder', 'dried basil', 'dried oregano', 'dried tarragon', 'dried thyme', 'powdered sugar', 'black pepper', 'cayenne pepper', 'red pepper flakes', 'celery seed']"
231634,zydeco ya ya deviled eggs,"['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'preparation', 'appetizers', 'eggs-dairy', 'easy', 'finger-food', 'eggs', 'presentation', 'served-cold', '3-steps-or-less']","['hard-cooked eggs', 'mayonnaise', 'dijon mustard', 'salt-free cajun seasoning', 'tabasco sauce', 'salt', 'black pepper', 'fresh italian parsley']"
231635,cookies by design cookies on a stick,"['30-minutes-or-less', 'time-to-make', 'course', 'preparation', 'for-large-groups', 'hand-formed-cookies', 'desserts', 'cookies-and-brownies', 'dietary', '

In [ ]:
df_recipes.shape

(231637, 3)

# Parsing the Ingredients

In [ ]:
!pip install unidecode

     |████████████████████████████████| 235 kB 5.2 MB/s 


In [ ]:
import pandas as pd
import nltk
import string
import ast
import re
import unidecode

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from collections import Counter

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
!touch dataset/parsed_recipes.csv

In [ ]:
def ingredient_parser(ingreds):
  # Words for removal

  words_to_remove = ["fresh","minced","chopped" "oil","a","red","bunch","and","clove","or","leaf","chilli","large","extra","sprig","ground","handful","free","small","pepper","virgin","range","from","dried","sustainable","black","peeled","higher","welfare","seed","for","finely","freshly","sea","quality","white","ripe","few","piece","source","to","organic","flat","smoked","ginger","sliced","green","picked","the","stick","plain","plus","mixed","mint","bay","basil","your","cumin","optional","fennel","serve","mustard","unsalted","baby","paprika","fat","ask","natural","skin","roughly","into","such","cut","good","brown","grated","trimmed","oregano","powder","yellow","dusting","knob","frozen","on","deseeded","low","runny","balsamic","cooked","streaky","nutmeg","sage","rasher","zest","pin","groundnut","breadcrumb","turmeric","halved","grating","stalk","light","tinned","dry","soft","rocket","bone","colour","washed","skinless","leftover","splash","removed","dijon","thick","big","hot","drained","sized","chestnut","watercress","fishmonger","english","dill","caper","raw","worcestershire","flake","cider","cayenne","tbsp","leg","pine","wild","if","fine","herb","almond","shoulder","cube","dressing","with","chunk","spice","thumb","garam","new","little","punnet","peppercorn","shelled","saffron","other" "chopped","salt","olive","taste","can","sauce","water","diced","package","italian","shredded","divided","parsley","vinegar","all","purpose","crushed","juice","more","coriander","bell","needed","thinly","boneless","half","thyme","cubed","cinnamon","cilantro","jar","seasoning","rosemary","extract","sweet","baking","beaten","heavy","seeded","tin","vanilla","uncooked","crumb","style","thin","nut","coarsely","spring","chili","cornstarch","strip","cardamom","rinsed","honey","cherry","root","quartered","head","softened","container","crumbled","frying","lean","cooking","roasted","warm","whipping","thawed","corn","pitted","sun","kosher","bite","toasted","lasagna","split","melted","degree","lengthwise","romano","packed","pod","anchovy","rom","prepared","juiced","fluid","floret","room","active","seasoned","mix","deveined","lightly","anise","thai","size","unsweetened","torn","wedge","sour","basmati","marinara","dark","temperature","garnish","bouillon","loaf","shell","reggiano","canola","parmigiano","round","canned","ghee","crust","long","broken","ketchup","bulk","cleaned","condensed","sherry","provolone","cold","soda","cottage","spray","tamarind","pecorino","shortening","part","bottle","sodium","cocoa","grain","french","roast","stem","link","firm","asafoetida","mild","dash","boiling","oil","chopped","vegetable oil","chopped oil","garlic","skin off","bone out" "from sustrainable sources",]

  if isinstance(ingreds, list):
        ingredients = ingreds
  else:
    ingredients = ast.literal_eval(ingreds)
    # We first get rid of all the punctuation. We make use of str.maketrans. It takes three input
    # arguments 'x', 'y', 'z'. 'x' and 'y' must be equal-length strings and characters in 'x'
    # are replaced by characters in 'y'. 'z' is a string (string.punctuation here) where each character
    #  in the string is mapped to None.
    translator = str.maketrans("", "", string.punctuation)
    lemmatizer = WordNetLemmatizer()
    ingred_list = []
    for i in ingredients:
      i.translate(translator)
      # We split up with hyphens as well as spaces
      items = re.split(" |-", i)
      # Get rid of words containing non alphabet letters
      items = [word for word in items if word.isalpha()]
      # Turn everything to lowercase
      items = [word.lower() for word in items]
      # remove accents
      items = [
          unidecode.unidecode(word) for word in items
      ]
      # Lemmatize words so we can compare words to measuring words
      items = [lemmatizer.lemmatize(word) for word in items]
      # Get rid of common easy words
      items = [word for word in items if word not in words_to_remove]
      if items:
        ingred_list.append(" ".join(items))
    return ingred_list
    
df_recipes["ingredients_parsed"] = df_recipes["ingredients"].apply(
    lambda x: ingredient_parser(x)
)
df_recipes.to_csv(r"dataset/parsed_recipes.csv", index=False)

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
df_recipes.head()

,name,tags,ingredients,ingredients_parsed
0,arriba baked winter squash mexican style,"['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'side-dishes', 'vegetables', 'mexican', 'easy', 'fall', 'holiday-event', 'vegetarian', 'winter', 'dietary', 'christmas', 'seasonal', 'squash']","['winter squash', 'mexican seasoning', 'mixed spice', 'honey', 'butter', 'olive oil', 'salt']","[winter squash, mexican, butter]"
1,a bit different breakfast pizza,"['30-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'breakfast', 'main-dish', 'pork', 'american', 'oven', 'easy', 'kid-friendly', 'pizza', 'dietary', 'northeastern-united-states', 'meat', 'equipment']","['prepared pizza crust', 'sausage patty', 'eggs', 'milk', 'salt and pepper', 'cheese']","[pizza, sausage patty, egg, milk, cheese]"
2,all in the kitchen chili,"['time-to-make', 'course', 'preparation', 'main-dish', 'chili', 'crock-pot-slow-cooker', 'dietary', 'equipment', '4-hours-or-less']","['ground beef', 'yellow onions', 'diced tomatoes', 'tomato paste', 'tomato soup', 'rotel tomatoes', 'kidney beans', 'water', 'chili powder', 'ground cumin', 'salt', 'lettuce', 'cheddar cheese']","[beef, onion, tomato, tomato paste, tomato soup, rotel tomato, kidney bean, lettuce, cheddar cheese]"
3,alouette potatoes,"['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'preparation', 'occasion', 'side-dishes', 'eggs-dairy', 'potatoes', 'vegetables', 'oven', 'easy', 'dinner-party', 'holiday-event', 'easter', 'cheese', 'stove-top', 'dietary', 'christmas', 'new-years', 'thanksgiving', 'independence-day', 'st-patricks-day', 'valentines-day', 'inexpensive', 'brunch', 'superbowl', 'equipment', 'presentation', 'served-hot']","['spreadable cheese with garlic and herbs', 'new potatoes', 'shallots', 'parsley', 'tarragon', 'olive oil', 'red wine vinegar', 'salt', 'pepper', 'red bell pepper', 'yellow bell pepper']","[spreadable cheese, potato, shallot, tarragon, wine]"
4,amish tomato ketchup for canning,"['weeknight', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'canning', 'condiments-etc', 'vegetables', 'american', 'heirloom-historical', 'holiday-event', 'vegetarian', 'dietary', 'amish-mennonite', 'northeastern-united-states', 'number-of-servings', 'technique', '4-hours-or-less']","['tomato juice', 'apple cider vinegar', 'sugar', 'salt', 'pepper', 'clove oil', 'cinnamon oil', 'dry mustard']","[tomato, apple, sugar]"


# RECCOMENDATION

In [ ]:
import os
import sys
import logging
import unidecode
import ast

import numpy as np
import pandas as pd

from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict

In [ ]:
def get_and_sort_corpus(data):
    """
    Get corpus with the documents sorted in alphabetical order
    """
    corpus_sorted = []
    for doc in data.ingredients_parsed.values:
        doc.sort()
        corpus_sorted.append(doc)
    return corpus_sorted

In [ ]:
def get_recommendations(N, scores):
    """
    Top-N recomendations order by score
    """
    # load in recipe dataset
    # df_recipes = pd.read_csv("dataset/RAW_recipes.csv")
    # order the scores with and filter to get the highest N scores
    top = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:N]
    # create dataframe to load in recommendations
    recommendation = pd.DataFrame(columns=["recipe", "ingredients", "score"])
    count = 0
    for i in top:
        recommendation.at[count, "recipe"] = title_parser(df_recipes["name"][i])
        recommendation.at[count, "ingredients"] = ingredient_parser_final(
            df_recipes["ingredients"][i]
        )
        recommendation.at[count, "score"] = f"{scores[i]}"
        count += 1
    return recommendation

In [ ]:
def title_parser(title):
    title = unidecode.unidecode(title)
    return title

In [ ]:
def ingredient_parser_final(ingredient):
    """
    neaten the ingredients being outputted
    """
    if isinstance(ingredient, list):
        ingredients = ingredient
    else:
        ingredients = ast.literal_eval(ingredient)

    ingredients = ",".join(ingredients)
    ingredients = unidecode.unidecode(ingredients)
    return ingredients

In [ ]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word_model):
        self.word_model = word_model
        self.vector_size = word_model.wv.vector_size

    def fit(self):  # comply with scikit-learn transformer requirement
        return self

    def transform(self, docs):  # comply with scikit-learn transformer requirement
        doc_word_vector = self.word_average_list(docs)
        return doc_word_vector

    def word_average(self, sent):
        """
		Compute average word vector for a single doc/sentence.
		:param sent: list of sentence tokens
		:return:
			mean: float of averaging word vectors
		"""
        mean = []
        for word in sent:
          if word in self.word_model.wv.index_to_key:
            mean.append(self.word_model.wv.get_vector(word))

        if not mean:  # empty words
            # If a text is empty, return a vector of zeros.
            return np.zeros(self.vector_size)
        else:
            mean = np.array(mean).mean(axis=0)
            return mean

    def word_average_list(self, docs):
      """
      Compute average word vector for multiple docs, where docs had been tokenized.
        :param docs: list of sentence in list of separated tokens
		    :return:
			    array of average word vector in shape (len(docs),)
      """
      return np.vstack([self.word_average(sent) for sent in docs])

In [ ]:
def get_recs(ingredients, N=5, mean=True):
    # load in word2vec model
    model = Word2Vec.load("models/model_cbow.bin")
    if model:
        print("Successfully loaded model")
    # load in data
    # df_parsed = pd.read_csv("dataset/parsed_recipes.csv")
    
    # parse ingredients
    df_recipes["ingredients_parsed"] = df_recipes["ingredients"].apply(
        lambda x: ingredient_parser(x)
    )

    # create corpus
    corpus = get_and_sort_corpus(df_recipes)

    if mean:
        # get average embdeddings for each document
        mean_vec_tr = MeanEmbeddingVectorizer(model)
        doc_vec = mean_vec_tr.transform(corpus)
        doc_vec = [doc.reshape(1, -1) for doc in doc_vec]

        assert len(doc_vec) == len(corpus)
    
    # create embessing for input text
    input1 = ingredients
    # create tokens with elements
    input1 = input1.split(",")
    
    input = []
    
    for i in input1:
      input.append(i)
    
    # print(input)
    df_input = pd.DataFrame(columns=['ingredients'])
    df_input = df_input.append({'ingredients': str(input)}, ignore_index=True)
    
    df_input["ingredients_parsed"] = df_input["ingredients"].apply(
        lambda x: ingredient_parser(x)
        # lambda x: print(type(x))
    )

    print(df_input.head())

    parsed_input = []
    for i in df_input['ingredients_parsed']:
      parsed_input = i
    print(parsed_input)

    if mean:
        input_embedding = mean_vec_tr.transform([parsed_input])[0].reshape(1, -1)
        # print(input_embedding)

    # get cosine similarity between input embedding and all the document embeddings
    cos_sim = map(lambda x: cosine_similarity(input_embedding, x)[0][0], doc_vec)
    scores = list(cos_sim)
    # print(scores
    # Filter top N recommendations
    recommendations = get_recommendations(N, scores)
    return recommendations

In [ ]:
corpus = get_and_sort_corpus(df_recipes)
print(f"Length of corpus: {len(corpus)}")

Length of corpus: 231637


In [ ]:
lengths = [len(doc) for doc in corpus]
avg_len = float(sum(lengths)) / len(lengths)
avg_len

5.898565427802985

In [ ]:
sg = 0 # CBOW: build a language model that correctly predicts the center word given the context words in which the center word appears
workers = 8 # number of CPUs
window = 6 # window size: average length of each document 
min_count = 1 # unique ingredients are important to decide recipes 

model_cbow = Word2Vec(corpus, sg=sg, workers=workers, window=window, min_count=min_count)

In [ ]:
print(model_cbow)
words = list(model_cbow.wv.index_to_key)
words.sort()

model_cbow.wv.most_similar(u'tomato')
# model_cbow.wv.similarity('capsicum', 'paste')

Word2Vec(vocab=8489, vector_size=100, alpha=0.025)


[('plum tomato', 0.8483703136444092),
 ('whole tomato', 0.7990044355392456),
 ('stewed tomato', 0.7356175780296326),
 ('tomato in puree', 0.7215704917907715),
 ('no added tomato', 0.6891029477119446),
 ('tomato puree', 0.6748018860816956),
 ('fire tomato', 0.6391302943229675),
 ('itailian essence', 0.6103074550628662),
 ('summer squash', 0.6053129434585571),
 ('garbanzo bean', 0.6041463613510132)]

In [ ]:
!mkdir models

In [ ]:
model_cbow.save('models/model_cbow.bin')

In [ ]:
df_recipes.head()

,name,tags,ingredients,ingredients_parsed
0,arriba baked winter squash mexican style,"['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'side-dishes', 'vegetables', 'mexican', 'easy', 'fall', 'holiday-event', 'vegetarian', 'winter', 'dietary', 'christmas', 'seasonal', 'squash']","['winter squash', 'mexican seasoning', 'mixed spice', 'honey', 'butter', 'olive oil', 'salt']","[butter, mexican, winter squash]"
1,a bit different breakfast pizza,"['30-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'breakfast', 'main-dish', 'pork', 'american', 'oven', 'easy', 'kid-friendly', 'pizza', 'dietary', 'northeastern-united-states', 'meat', 'equipment']","['prepared pizza crust', 'sausage patty', 'eggs', 'milk', 'salt and pepper', 'cheese']","[cheese, egg, milk, pizza, sausage patty]"
2,all in the kitchen chili,"['time-to-make', 'course', 'preparation', 'main-dish', 'chili', 'crock-pot-slow-cooker', 'dietary', 'equipment', '4-hours-or-less']","['ground beef', 'yellow onions', 'diced tomatoes', 'tomato paste', 'tomato soup', 'rotel tomatoes', 'kidney beans', 'water', 'chili powder', 'ground cumin', 'salt', 'lettuce', 'cheddar cheese']","[beef, cheddar cheese, kidney bean, lettuce, onion, rotel tomato, tomato, tomato paste, tomato soup]"
3,alouette potatoes,"['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'preparation', 'occasion', 'side-dishes', 'eggs-dairy', 'potatoes', 'vegetables', 'oven', 'easy', 'dinner-party', 'holiday-event', 'easter', 'cheese', 'stove-top', 'dietary', 'christmas', 'new-years', 'thanksgiving', 'independence-day', 'st-patricks-day', 'valentines-day', 'inexpensive', 'brunch', 'superbowl', 'equipment', 'presentation', 'served-hot']","['spreadable cheese with garlic and herbs', 'new potatoes', 'shallots', 'parsley', 'tarragon', 'olive oil', 'red wine vinegar', 'salt', 'pepper', 'red bell pepper', 'yellow bell pepper']","[potato, shallot, spreadable cheese, tarragon, wine]"
4,amish tomato ketchup for canning,"['weeknight', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'canning', 'condiments-etc', 'vegetables', 'american', 'heirloom-historical', 'holiday-event', 'vegetarian', 'dietary', 'amish-mennonite', 'northeastern-united-states', 'number-of-servings', 'technique', '4-hours-or-less']","['tomato juice', 'apple cider vinegar', 'sugar', 'salt', 'pepper', 'clove oil', 'cinnamon oil', 'dry mustard']","[apple, sugar, tomato]"


In [ ]:
input = ""
rec = get_recs(input, 10)
print(rec)

Successfully loaded model
  ingredients ingredients_parsed
0        ['']                 []
[]
                                       recipe  \
0  arriba   baked winter squash mexican style   
1            a bit different  breakfast pizza   
2                   all in the kitchen  chili   
3                          alouette  potatoes   
4          amish  tomato ketchup  for canning   
5                     apple a day  milk shake   
6                       aww  marinated olives   
7              backyard style  barbecued ribs   
8                    bananas 4 ice cream  pie   
9                     beat this  banana bread   

                                                                                                                                                                                                                                                                                                  ingredients  \
0                                                           

In [ ]:
ingreds = ["fresh tomato", "apple", "sugar"]

# print(type(input))
# df = pd.DataFrame(columns=["ingredients"])
# for i in ingreds:
#   print(type(i))
#   df = df.append({'ingredients': str(i)}, ignore_index=True)
#   print("done")
# df['parsed_ingredients'] = df['ingredients'].apply(
#     lambda x: ingredient_parser(x)
# )

<class 'str'>
done
<class 'str'>
done
<class 'str'>
done


In [ ]:
df.head()

,ingredients
0,fresh tomato
1,apple
2,sugar


## Filtering code(trial) - NOT FOR RUNNING ATM

In [ ]:
found = 0
for i in df_recipes['tags']:
  found+=1
  if(not "ham" in i):
    df_recipes = df_recipes.drop(labels=(found-1), axis=0)
    # print(found-1)


In [ ]:
# ['cream of tartar', 'egg', 'flour', 'granulated sugar']
found = 0
for i in df_recipes['tags']:
  # found+=1
  if(not "vegetarian" in i):
    found+=1
    # print(df_recipes.loc[[23636]])
    # df_recipes = df_recipes.drop(labels=(found-1), axis=0)
print(found)

# print(df_recipes.loc[[231636]])

195986


In [ ]:
df_recipes.shape

(231304, 4)

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
print(df_recipes.loc[[115093]])

                                        name  \
115093  jewish chicken soup with matzo balls   

                                                                                                                                     tags  \
115093  ['time-to-make', 'course', 'cuisine', 'preparation', 'soups-stews', 'jewish-ashkenazi', 'european', 'dietary', '4-hours-or-less']   

                                                                                                                                                                ingredients  \
115093  ['eggs', 'egg white', 'salt', 'matzo meal', 'oil', 'homemade chicken broth', 'parsnips', 'carrot', 'onion', 'broccoli floret', 'mushroom', 'fresh dill', 'parsley']   

                                                                                ingredients_parsed  
115093  [broccoli, carrot, egg, egg, homemade chicken broth, matzo meal, mushroom, onion, parsnip]  


In [ ]:
!unzip food-com-recipes-and-user-interactions.zip -d dataset/

Archive:  food-com-recipes-and-user-interactions.zip
replace dataset/PP_recipes.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dataset/PP_users.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dataset/RAW_interactions.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dataset/RAW_recipes.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: dataset/RAW_recipes.csv  
replace dataset/ingr_map.pkl? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dataset/interactions_test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dataset/interactions_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dataset/interactions_validation.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
import pandas as pd
df_recipes = pd.read_csv("dataset/RAW_recipes.csv")
df_recipes = df_recipes.drop(columns = ['minutes', 'id', 'contributor_id', 'submitted', 'nutrition', 'n_steps', 'steps', 'description', 'n_ingredients'])

In [ ]:
df_recipes = df_recipes.drop(range(220054,231636))

In [ ]:
df_recipes.shape

(220055, 3)

In [ ]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [ ]:
df_recipes

,name,tags,ingredients
0,arriba baked winter squash mexican style,"['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'side-dishes', 'vegetables', 'mexican', 'easy', 'fall', 'holiday-event', 'vegetarian', 'winter', 'dietary', 'christmas', 'seasonal', 'squash']","['winter squash', 'mexican seasoning', 'mixed spice', 'honey', 'butter', 'olive oil', 'salt']"
1,a bit different breakfast pizza,"['30-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'breakfast', 'main-dish', 'pork', 'american', 'oven', 'easy', 'kid-friendly', 'pizza', 'dietary', 'northeastern-united-states', 'meat', 'equipment']","['prepared pizza crust', 'sausage patty', 'eggs', 'milk', 'salt and pepper', 'cheese']"
2,all in the kitchen chili,"['time-to-make', 'course', 'preparation', 'main-dish', 'chili', 'crock-pot-slow-cooker', 'dietary', 'equipment', '4-hours-or-less']","['ground beef', 'yellow onions', 'diced tomatoes', 'tomato paste', 'tomato soup', 'rotel tomatoes', 'kidney beans', 'water', 'chili powder', 'ground cumin', 'salt', 'lettuce', 'cheddar cheese']"
3,alouette potatoes,"['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'preparation', 'occasion', 'side-dishes', 'eggs-dairy', 'potatoes', 'vegetables', 'oven', 'easy', 'dinner-party', 'holiday-event', 'easter', 'cheese', 'stove-top', 'dietary', 'christmas', 'new-years', 'thanksgiving', 'independence-day', 'st-patricks-day', 'valentines-day', 'inexpensive', 'brunch', 'superbowl', 'equipment', 'presentation', 'served-hot']","['spreadable cheese with garlic and herbs', 'new potatoes', 'shallots', 'parsley', 'tarragon', 'olive oil', 'red wine vinegar', 'salt', 'pepper', 'red bell pepper', 'yellow bell pepper']"
4,amish tomato ketchup for canning,"['weeknight', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'canning', 'condiments-etc', 'vegetables', 'american', 'heirloom-historical', 'holiday-event', 'vegetarian', 'dietary', 'amish-mennonite', 'northeastern-united-states', 'number-of-servings', 'technique', '4-hours-or-less']","['tomato juice', 'apple cider vinegar', 'sugar', 'salt', 'pepper', 'clove oil', 'cinnamon oil', 'dry mustard']"
...,...,...,...
162141,pollo alla toscana,"['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'main-dish', 'poultry', 'european', 'italian', 'chicken', 'meat']","['boneless skinless chicken thighs', 'chicken tenderloins', 'salt and pepper', 'extra virgin olive oil', 'garlic cloves', 'white wine vinegar', 'butter', 'shallots', 'fresh rosemary', 'flour', 'dry white wine', 'beef broth']"
162142,pollo asada burrito,"['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'north-american', 'main-dish', 'poultry', 'mexican', 'easy', 'chicken', 'dietary', 'meat']","['chicken breasts', 'avocados', 'tomatoes', 'onions', 'lemon juice', 'sea salt', 'garlic clove', 'cilantro', 'sour cream', 'cheddar cheese', 'flour tortillas']"
162143,pollo asado,"['60-minutes-or-less', 'time-to-make', 'preparation']","['olive oil', 'ground cumin', 'salt', 'dried oregano', 'garlic', 'lime juice', 'orange juice', 'achiote powder', 'whole boneless chicken breasts']"
162144,pollo asado con llajua cumin grilled chicken with spicy salsa,"['time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'very-low-carbs', 'main-dish', 'poultry', 'chicken', 'south-american', 'dietary', 'low-carb', 'low-in-something', 'meat', 'chicken-breasts', 'grilling', 'technique', '4-hours-or-less']","['hot pepper', 'tomatoes', 'garlic clove', 'olive oil', 'red wine vinegar', 'coarse salt', 'pepper', 'fresh cilantro', 'chicken breast halves', 'black pepper', 'ground cumin']"


In [ ]:
df_recipes.to_csv("dataset/reduced_recipe.csv")